# Diagrama de Sankey
---

En esta celda, vamos a utilizar los datos de esta primera iteración del modelado para analizar como se dividió todo el flujo de estudiantes del curso entre la entrega de tareas de evaluación continua del curso y respecto a las notas que obtuvieron

## Diagrama de entregas


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Assuming DATA_PATH and df_entregas setup from your original code
DATA_PATH = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"
df_entregas = pd.read_parquet(f"{DATA_PATH}/dataset_1.1.parquet")

df_entregas = df_entregas.drop(columns=["num_entregas"])

columns_entregas = df_entregas.columns.tolist()
columns_entregas.remove("Test Complejidad")
columns_entregas.insert(-2, "Test Complejidad")
df_entregas = df_entregas[columns_entregas]

columns_entregas.remove("abandona")
columns_entregas.append("abandona")
df_entregas = df_entregas[columns_entregas]

# ─────────────────── FUNCIONES ────────────────────
def build_sankey_entregas_abandono(df):
    """
    Crea nodos, enlaces y colores para:
      actividades 0/1  →  'abandona' (0 = Sigue, 1 = Abandona)
    """
    # Excluye 'userid' y 'abandona' de las columnas de actividad iniciales
    activity_cols = [c for c in df.columns if c not in {"userid", "abandona"}]
    
    node_idx, labels = {}, []
    source, target, value, link_colors = [], [], [], []
    
    # helper: traduce valor a etiqueta según la columna
    def label(col, val):
        if col == "abandona":
            return "Abandona" if val == 1 else "Sigue"
        return "Entregó" if val == 1 else "No entregó"
    
    # Define colores para los enlaces
    color_abandona = "rgba(255, 0, 0, 0.4)"  # Rojo para abandono
    color_sigue = "rgba(0, 128, 0, 0.4)"     # Verde para continuar
    
    # 1) registrar nodos para las actividades
    for col in activity_cols:
        for val in (1, 0): # 1 primero para que quede arriba
            node_idx[(col, val)] = len(labels)
            labels.append(f"{col}<br>{label(col,val)}")
            
    # 1.1) registrar nodos para el estado final de abandono
    # Estos nodos deben aparecer al final del diagrama.
    # Necesitamos asegurar que sus índices sean únicos y secuenciales después de las actividades.
    # Las etiquetas "Sigue" y "Abandona" se añadirán una vez al final.
    node_idx[("abandona", 0)] = len(labels) # Sigue
    labels.append("Sigue")
    node_idx[("abandona", 1)] = len(labels) # Abandona
    labels.append("Abandona")

    # 2) contar transiciones entre columnas consecutivas de actividades
    for col1, col2 in zip(activity_cols[:-1], activity_cols[1:]):
        counts = (
            df.groupby([col1, col2, "abandona"]).size().reset_index(name="n")
        )
        for _, row in counts.iterrows():
            v1, v2, abandona_status, n = int(row[col1]), int(row[col2]), int(row["abandona"]), int(row["n"])
            
            source.append(node_idx[(col1, v1)])
            target.append(node_idx[(col2, v2)])
            value.append(n)
            
            if abandona_status == 1:
                link_colors.append(color_abandona)
            else:
                link_colors.append(color_sigue)

    # 3) Contar transiciones de la última actividad al estado final (abandona/sigue)
    last_activity_col = activity_cols[-1]
    counts_to_final = (
        df.groupby([last_activity_col, "abandona"]).size().reset_index(name="n")
    )
    for _, row in counts_to_final.iterrows():
        v_last_act, abandona_status, n = int(row[last_activity_col]), int(row["abandona"]), int(row["n"])
        
        source.append(node_idx[(last_activity_col, v_last_act)])
        target.append(node_idx[("abandona", abandona_status)]) # Conecta con el nodo 'Abandona' o 'Sigue'
        value.append(n)
        
        if abandona_status == 1:
            link_colors.append(color_abandona)
        else:
            link_colors.append(color_sigue)
    
    return labels, source, target, value, link_colors

def plot_sankey(labels, source, target, value, link_colors, title):
    fig = go.Figure(go.Sankey(
        arrangement="snap",
        node=dict(label=labels, pad=15, thickness=16,
                  color="rgba(44,160,101,0.8)"), # El color del nodo puede ser genérico o personalizado
        link=dict(source=source, target=target, value=value,
                  color=link_colors) # Usar los colores de los enlaces generados dinámicamente
    ))
    fig.update_layout(title_text=title, font_size=12, height=600)
    fig.show()

# ─────────────────── EJECUCIÓN ────────────────────
labels, src, tgt, val, link_colors = build_sankey_entregas_abandono(df_entregas)

plot_sankey(labels, src, tgt, val, link_colors,
            "Flujo de ENTREGAS por actividad ➜ Estado final (Abandona / Sigue)")